# Doc2Vec model
> * Positive or Negative

In [7]:
import pickle

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [8]:
import multiprocessing
cores = multiprocessing.cpu_count()
def Make_Doc2Vec_Model(data, size, dm, dm_concat, dm_mean, hs, negative, epoch, window, alpha, min_alpha, workers, tagger):
    from tqdm import tqdm
    tqdm.pandas(desc="progress-bar")
    from datetime import datetime
    from gensim.models import doc2vec
    start = datetime.now()
    modelPath = './model/'
    modelName = 'doc2vec_size-{}_epoch-{}_window-{}_negative-{}_hs-{}_dm-{}_dm_concat-{}_dm_mean-{}_by-{}.model'.format(
        size, epoch, window, negative, hs, dm, dm_concat, dm_mean, tagger)
    modelName = modelPath+modelName
    print (modelName)
    d2v_model = doc2vec.Doc2Vec(vector_size = size, dm = dm, dm_concat = dm_concat,
                   dm_mean = dm_mean, negative = negative, hs = hs, window = window,
                   alpha = alpha, min_alpha = min_alpha, workers = workers, epochs= epoch)
    d2v_model.build_vocab(tqdm(data))
    d2v_model.train(tqdm(train), total_examples=d2v_model.corpus_count, epochs=d2v_model.iter)
    
    end = datetime.now()
    d2v_model.save(modelName)
    print ("Total running time: ", end-start)
    return d2v_model

# Doc2Vec 생성

* vector size
 [ 1000, 2000 ]

In [ ]:
import numpy as np
import pandas as pd

In [5]:
rawdata = pd.read_csv('./data/sentiment_data/raw_data_for_sentiment.txt',header=None,encoding='utf-8')
print (rawdata.shape)

(491510, 2)


### tagging twitter

In [ ]:
from ckonlpy.tag import Twitter as ctwitter
ct = ctwitter()

In [8]:
# twitter
def tokenize1(doc):
    return ['/'.join(t) for t in ct.pos(doc)]

In [14]:
# pickle로 저장된 파일이 없을 때
raw_doc_ct = [(tokenize1(rawdata.loc[idx][0]), rawdata.loc[idx][1]) for idx in tqdm(rawdata.index)]
pickle.dump(raw_doc_ct, open('./data/pre_data/tagged_data/pre_data_by_ct_for_sentiment_analysis.pickled','wb'))

In [15]:
# pickle로 저장된 파일이 없을 때
tagged_ct = [TaggedDocument(d, [c]) for d, c in tqdm(raw_doc_ct)]
pickle.dump(tagged_ct, open('./data/pre_data/tagged_data/pre_by_ct_data_tagged_run_docs.pickled','wb'))

In [16]:
del raw_doc_ct

#### model 만들기

In [ ]:
from collections import namedtuple
from gensim.models.doc2vec import TaggedDocument
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [ ]:
# pickle로 저장된 파일이 있을 때
tagged_ct = pickle.load(open('./data/pre_data/tagged_data/pre_by_ct_data_tagged_run_docs.pickled','rb'))

In [4]:
from sklearn.model_selection import train_test_split

In [17]:
# pickle로 저장된 파일이 없을 때
train, test = train_test_split(tagged_ct, test_size=0.1, random_state=42)
del tagged_ct
pickle.dump(train, open('./data/pre_data/train_test_Data/pre_by_ct_train.pickled','wb'))
pickle.dump(test, open('./data/pre_data/train_test_Data/pre_by_ct_test.pickled','wb'))

In [6]:
# pickle로 저장된 파일이 있을 때
train = pickle.load(open('./data/pre_data/train_test_Data/pre_by_ct_train.pickled','rb'))
test = pickle.load(open('./data/pre_data/train_test_Data/pre_by_ct_test.pickled','rb'))

In [ ]:
from konlpy.utils import pprint

In [ ]:
%%time
#PV-DM W/
d2v_model = Make_Doc2Vec_Model(data=train, size = 2000, dm = 1, dm_concat = 1,
                   dm_mean = 0, negative = 7, hs = 0, epoch = 20, window = 5,
                   alpha = 0.025, min_alpha = 0.025, workers = cores, tagger = 'ct')
pprint(d2v_model.most_similar('문재인/Noun'))
pprint(d2v_model.most_similar('노무현/Noun'))
pprint(d2v_model.most_similar('박근혜/Noun'))

In [ ]:
del d2v_model

In [ ]:
%%time
#PV-DBOW
d2v_model = Make_Doc2Vec_Model(data=train, size = 2000, dm = 1, dm_concat = 0,
                   dm_mean = 1, negative = 7, hs = 0, epoch = 20, window = 10,
                   alpha = 0.025, min_alpha = 0.025, workers = cores, tagger = 'ct')
pprint(d2v_model.most_similar('문재인/Noun'))
pprint(d2v_model.most_similar('노무현/Noun'))
pprint(d2v_model.most_similar('박근혜/Noun'))

In [ ]:
del d2v_model

In [ ]:
%%time
#PV-DM w/
d2v_model = Make_Doc2Vec_Model(data=train, size = 2000, dm = 0, dm_concat = 0,
                   dm_mean = 0, negative = 7, hs = 0, epoch = 20, window = 5,
                   alpha = 0.025, min_alpha = 0.025, workers = cores, tagger = 'ct')
pprint(d2v_model.most_similar('문재인/Noun'))
pprint(d2v_model.most_similar('노무현/Noun'))
pprint(d2v_model.most_similar('박근혜/Noun'))

In [9]:
del train
del test
del d2v_model

### tagging mecab

In [23]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
# mecab
def tokenize2(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

In [24]:
# pickle로 저장된 파일이 없을 때
raw_doc_mecab = [(tokenize2(rawdata.loc[idx][0]), rawdata.loc[idx][1]) for idx in tqdm(rawdata.index)]
pickle.dump(raw_doc_mecab, open('./data/pre_data/tagged_data/pre_data_by_mecab_for_sentiment_analysis.pickled','wb'))

In [25]:
# pickle로 저장된 파일이 없을 때
tagged_mecab = [TaggedDocument(d, [c]) for d, c in raw_doc_mecab]
pickle.dump(tagged_mecab, open('./data/pre_data/tagged_data/pre_by_mecab_data_tagged_run_docs.pickled','wb'))

In [26]:
del raw_doc_mecab

#### model 만들기

In [ ]:
from collections import namedtuple
from gensim.models.doc2vec import TaggedDocument
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [ ]:
# pickle로 저장된 파일이 있을 때
tagged_mecab = pickle.load(open('./data/pre_data/tagged_data/pre_by_mecab_data_tagged_run_docs.pickled','rb'))

In [4]:
from sklearn.model_selection import train_test_split

In [17]:
# pickle로 저장된 파일이 없을 때
train2, test2 = train_test_split(tagged_mecab, test_size=0.1, random_state=42)
del tagged_ct
pickle.dump(train2, open('./data/pre_data/train_test_Data/pre_by_mecab_train.pickled','wb'))
pickle.dump(test2, open('./data/pre_data/train_test_Data/pre_by_mecab_test.pickled','wb'))

In [6]:
# pickle로 저장된 파일이 있을 때
train2 = pickle.load(open('./data/pre_data/train_test_Data/pre_by_mecab_train.pickled','rb'))
test2 = pickle.load(open('./data/pre_data/train_test_Data/pre_by_mecab_test.pickled','rb'))

In [ ]:
from konlpy.utils import pprint

In [ ]:
%%time
#PV-DM W/
d2v_model = Make_Doc2Vec_Model(data=train2, size = 2000, dm = 1, dm_concat = 1,
                   dm_mean = 0, negative = 7, hs = 0, epoch = 20, window = 5,
                   alpha = 0.025, min_alpha = 0.025, workers = cores, tagger = 'mecab')
pprint(d2v_model.most_similar('문재인/NNP'))
pprint(d2v_model.most_similar('노무현/NNP'))
pprint(d2v_model.most_similar('박근혜/NNP'))

In [ ]:
del d2v_model

In [ ]:
%%time
#PV-DBOW
d2v_model = Make_Doc2Vec_Model(data=train2, size = 2000, dm = 1, dm_concat = 0,
                   dm_mean = 1, negative = 7, hs = 0, epoch = 20, window = 10,
                   alpha = 0.025, min_alpha = 0.025, workers = cores, tagger = 'mecab')
pprint(d2v_model.most_similar('문재인/NNP'))
pprint(d2v_model.most_similar('노무현/NNP'))
pprint(d2v_model.most_similar('박근혜/NNP'))

In [ ]:
del d2v_model

In [ ]:
%%time
#PV-DM w/
d2v_model = Make_Doc2Vec_Model(data=train2, size = 2000, dm = 0, dm_concat = 0,
                   dm_mean = 0, negative = 7, hs = 0, epoch = 20, window = 5,
                   alpha = 0.025, min_alpha = 0.025, workers = cores, tagger = 'mecab')
pprint(d2v_model.most_similar('문재인/NNP'))
pprint(d2v_model.most_similar('노무현/NNP'))
pprint(d2v_model.most_similar('박근혜/NNP'))

In [9]:
del train
del test
del d2v_model